### Goal
Analyze raw order data to compute core revenue quality metrics after cleaning and validation.

In [1]:
!pip install duckdb

In [2]:
import duckdb
import pandas as pd

In [3]:
orders = pd.read_csv("data/orders_raw.csv")

In [4]:
orders.head()

,order_id,user_id,email_hash,order_ts_raw,amount_raw,currency,status
0,ORD-14940,95.0,fe6e8699b47aced6c8f30f7f94f7ef8d,02/01/2024 02:17,currency 98.61,currency,PAID
1,ORD-28995,285.0,70f98e0a5b65109772648f23b5a01147,2024-02-09,229.6,د.إ,Paid
2,ORD-10795,NaN,NaN,2024-01-28 05:49:51,aed 186.94,aed,COMPLETE
3,ORD-36156,104.0,6d25f5051bb9436ef042e2fdde2f0650,2024/05/14 22:39 GMT,free,currency,COMPLETE
4,ORD-96564,335.0,62666207dbaa3b80af7949ec7722372f,04/03/2024 15:11,106.07,usd,PAID


In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      267 non-null    object 
 1   user_id       229 non-null    float64
 2   email_hash    213 non-null    object 
 3   order_ts_raw  265 non-null    object 
 4   amount_raw    267 non-null    object 
 5   currency      227 non-null    object 
 6   status        267 non-null    object 
dtypes: float64(1), object(6)
memory usage: 14.7+ KB


In [6]:
con = duckdb.connect()

In [7]:
con.register("orders_raw", orders)

### Audit 1 - Order status inventory

In [8]:
con.execute("""SELECT status, COUNT(*) AS n FROM orders_raw
GROUP BY status
ORDER BY n DESC""").df()

,status,n
0,COMPLETE,44
1,Paid,42
2,paid,40
3,SUCCESS,40
4,completed,38
5,success,35
6,PAID,28


All observed statuses represent successful orders and will be normalized into a single canonical status: paid.

In [9]:
con.execute("""SELECT *,
CASE
WHEN LOWER(status) IN ('complete', 'completed', 'paid', 'success') 
THEN 'paid'
ELSE 'unknown'
END AS status_clean
FROM orders_raw""").df()

,order_id,user_id,email_hash,order_ts_raw,amount_raw,currency,status,status_clean
0,ORD-14940,95.0,fe6e8699b47aced6c8f30f7f94f7ef8d,02/01/2024 02:17,currency 98.61,currency,PAID,paid
1,ORD-28995,285.0,70f98e0a5b65109772648f23b5a01147,2024-02-09,229.6,د.إ,Paid,paid
2,ORD-10795,NaN,None,2024-01-28 05:49:51,aed 186.94,aed,COMPLETE,paid
3,ORD-36156,104.0,6d25f5051bb9436ef042e2fdde2f0650,2024/05/14 22:39 GMT,free,currency,COMPLETE,paid
4,ORD-96564,335.0,62666207dbaa3b80af7949ec7722372f,04/03/2024 15:11,106.07,usd,PAID,paid
...,...,...,...,...,...,...,...,...
262,ORD-31544,426.0,1d22088823b0f562a727f56984b9e40d,02/11/2024 18:52,82.19,AED,SUCCESS,paid
263,ORD-81739,NaN,None,None,237.36,AED,PAID,paid
264,ORD-96424,311.0,da9d56e411e0def651fbe4797b49bb8e,2024/01/17 12:47,currency 149.78,currency,completed,paid
265,ORD-85213,424.0,cca60d7068fcc43d4487f73682b28cdb,2024-04-23,70.06,US$,success,paid


In [10]:
con.execute("""
SELECT status_clean, COUNT(*) AS n
FROM (
SELECT
CASE
WHEN LOWER(status) IN ('complete','completed','paid','success')
THEN 'paid'
ELSE 'unknown'
END AS status_clean
FROM orders_raw)
GROUP BY status_clean""").df()

,status_clean,n
0,paid,267


### Audit 2 - amount_raw inventory

In [11]:
con.execute("""
SELECT amount_raw, COUNT(*) AS n
FROM orders_raw
GROUP BY amount_raw
ORDER BY n DESC
LIMIT 20""").df()

,amount_raw,n
0,free,11
1,162.16,2
2,70.72,2
3,93.19,1
4,16.09,1
5,86.46,1
6,145.48,1
7,aed 186.94,1
8,219.56,1
9,68.93,1


Parse numeric values from strings, treat free as 0.0, anything unparseable, null

In [12]:
con.execute("""
SELECT amount_raw,
CASE
WHEN LOWER(TRIM(amount_raw)) = 'free' THEN 0
ELSE
TRY_CAST(REGEXP_EXTRACT(TRIM(amount_raw), '(-?\\d+(?:\\.\\d+)?)', 1) AS DOUBLE)
END AS amount_clean
FROM orders_raw
LIMIT 30;
""").df()

,amount_raw,amount_clean
0,currency 98.61,98.61
1,229.6,229.60
2,aed 186.94,186.94
3,free,0.00
4,106.07,106.07
5,195.8,195.80
6,81.8,81.80
7,62.16,62.16
8,aed 111.65,111.65
9,93.19,93.19


### Audit 3 - currency inventory

In [13]:
con.execute("""SELECT currency, COUNT(*) AS n FROM orders_raw
GROUP BY currency
ORDER BY n DESC""").df()

,currency,n
0,None,40
1,AED,39
2,aed,35
3,usd,34
4,currency,33
5,US$,33
6,د.إ,30
7,USD,23


In [14]:
con.execute("""SELECT amount_raw, currency,
CASE
WHEN currency IS NOT NULL AND LOWER(TRIM(currency)) IN ('aed', 'د.إ') THEN 'AED'
WHEN currency IS NOT NULL AND LOWER(TRIM(currency)) IN ('usd', 'us$') THEN 'USD'
WHEN currency IS NULL OR LOWER(TRIM(currency)) = 'currency' THEN
CASE
WHEN LOWER(TRIM(amount_raw)) LIKE '%usd%' OR 
LOWER(TRIM(amount_raw)) LIKE '%us$%' THEN 'USD'
WHEN LOWER(amount_raw) LIKE '%aed%' OR TRIM(amount_raw) LIKE '%د.إ%' THEN 'AED'
ELSE NULL
END
ELSE NULL
END AS currency_clean
FROM orders_raw
LIMIT 50;""").df()

,amount_raw,currency,currency_clean
0,currency 98.61,currency,None
1,229.6,د.إ,AED
2,aed 186.94,aed,AED
3,free,currency,None
4,106.07,usd,USD
5,195.8,US$,USD
6,81.8,US$,USD
7,62.16,US$,USD
8,aed 111.65,aed,AED
9,93.19,د.إ,AED


In [15]:
con.execute("""
WITH tmp AS (SELECT
CASE
WHEN currency IS NOT NULL AND LOWER(TRIM(currency)) IN ('aed', 'د.إ') THEN 'AED'
WHEN currency IS NOT NULL AND LOWER(TRIM(currency)) IN ('usd', 'us$') THEN 'USD'
WHEN currency IS NULL OR LOWER(TRIM(currency)) = 'currency' THEN
CASE
WHEN LOWER(TRIM(amount_raw)) LIKE '%usd%' OR 
LOWER(TRIM(amount_raw)) LIKE '%us$%' THEN 'USD'
WHEN LOWER(amount_raw) LIKE '%aed%' OR TRIM(amount_raw) LIKE '%د.إ%' THEN 'AED'
ELSE NULL
END
ELSE NULL
END AS currency_clean
FROM orders_raw)
SELECT SUM(CASE WHEN currency_clean IS NULL THEN 1 ELSE 0 END) AS null_currency_clean
FROM tmp;""").df()

,null_currency_clean
0,69.0


In [16]:
con.execute("""
CREATE OR REPLACE VIEW clean_orders AS 
SELECT order_id, user_id, email_hash, order_ts_raw,
CASE
WHEN LOWER(TRIM(amount_raw)) = 'free' THEN 0
ELSE
TRY_CAST(REGEXP_EXTRACT(TRIM(amount_raw), '(-?\\d+(?:\\.\\d+)?)', 1) AS DOUBLE)
END AS amount_clean,
amount_raw,
CASE
WHEN currency IS NOT NULL AND LOWER(TRIM(currency)) IN ('aed', 'د.إ') THEN 'AED'
WHEN currency IS NOT NULL AND LOWER(TRIM(currency)) IN ('usd', 'us$') THEN 'USD'
WHEN currency IS NULL OR LOWER(TRIM(currency)) = 'currency' THEN
CASE
WHEN LOWER(TRIM(amount_raw)) LIKE '%usd%' OR 
LOWER(TRIM(amount_raw)) LIKE '%us$%' THEN 'USD'
WHEN LOWER(amount_raw) LIKE '%aed%' OR TRIM(amount_raw) LIKE '%د.إ%' THEN 'AED'
ELSE NULL
END
ELSE NULL
END AS currency_clean,
currency,
CASE
WHEN LOWER(status) IN ('complete','completed','paid','success')
THEN 'paid'
ELSE 'unknown'
END AS status_clean,
status,
CASE
WHEN LOWER(TRIM(amount_raw)) <> 'free'
AND TRY_CAST(REGEXP_EXTRACT(TRIM(amount_raw), '(-?\\d+(?:\\.\\d+)?)', 1) AS DOUBLE) IS NULL
THEN TRUE ELSE FALSE
END AS flag_bad_amount,
CASE
WHEN currency_clean IS NULL THEN TRUE 
ELSE FALSE
END AS flag_missing_currency
FROM orders_raw;""")

Sanity Checks

In [17]:
con.execute("SELECT COUNT(*) FROM clean_orders").df()

,count_star()
0,267


In [18]:
con.execute("""SELECT SUM(flag_bad_amount::INT) AS bad_amounts
FROM clean_orders""").df()

,bad_amounts
0,0.0


In [19]:
con.execute("""SELECT currency_clean, COUNT(*) AS n FROM clean_orders
GROUP BY currency_clean
ORDER BY n DESC""").df()

,currency_clean,n
0,AED,104
1,USD,94
2,None,69


### Final Metrics

Total Revenue by Currency

In [20]:
con.execute("""SELECT currency_clean, ROUND(SUM(amount_clean), 2) AS total_revenue
FROM clean_orders
WHERE status_clean = 'paid'
GROUP BY currency_clean
ORDER BY total_revenue DESC""").df()

,currency_clean,total_revenue
0,AED,13630.25
1,USD,9946.20
2,None,8197.71


Average Order Value (AOV)

In [21]:
con.execute("""SELECT currency_clean, ROUND(AVG(amount_clean), 2) AS aov
FROM clean_orders
WHERE status_clean = 'paid'
GROUP BY currency_clean""").df()

,currency_clean,aov
0,None,118.81
1,AED,131.06
2,USD,105.81


### Conclusion

Out of 267 total orders, all observed statuses mapped to successful payments, while approximately 26% of orders lacked an explicit currency and were treated separately to avoid mixing revenue units.
This analysis highlights the importance of isolating transactional integrity from behavioral funnels and demonstrates how careful data cleaning enables reliable revenue metrics such as total revenue and average order value.